In [ ]:
# Author: Joseph Salmon <joseph.salmon@telecom-paristech.fr>
#         Mathurin Massias <mathurin.massias@gmail.com>

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import check_random_state
# make sure the share_code package has been installed 
# with `pip install -e .` at the root of the repo
from share_code.utils import make_huber, my_saving_display

sns.set_palette("colorblind")
%matplotlib inline 

In [ ]:
dirname = "../prebuiltimages/"
imageformat = ".pdf"  # should be .pdf or .png    
# some colors I'll use
brown = (0.64, 0.16, 0.16)
purple = (148. / 255, 0, 211. / 255)
plt.close("all")
saving = False  # True

In [ ]:
color_blind_list = sns.color_palette("GnBu_d", 8)
c1 = color_blind_list[0]
c2 = color_blind_list[1]
c3 = color_blind_list[2]

In [ ]:
def gradient_descent(func, func_prime, steps, theta_init, n_iter=10):
    if not isinstance(steps, np.ndarray):
        steps = np.repeat(steps, n_iter)

    theta_init = np.atleast_1d(theta_init).astype(np.float64)
    all_thetas = np.zeros([n_iter, theta_init.shape[0]])

    all_objs = np.zeros([n_iter])

    theta = theta_init.copy()

    for it in range(n_iter):
        theta -= steps[it] * func_prime(theta)
        all_thetas[it] = theta
        all_objs[it] = func(theta)
    return all_thetas, all_objs

# Geometric Median case

In [ ]:
rng = check_random_state(24)
n_samples, n_features = 5, 1

X = rng.normal(0, 1, size=(n_samples, n_features))
n_iter = 50
theta_init = np.mean(X, axis=0)

In [ ]:
# thresh = 0.1
thresh = 0.3

huber, huber_prime = make_huber(thresh)

def cost_median(theta):
    return np.linalg.norm(X - theta[None, :], axis=1).sum()


def cost_huber(theta):
    cost = huber(X[:, 0] - theta).sum()
    return cost


def cost_huber_prime(theta):
    return - huber_prime(X[:, 0] - theta).sum(axis=0)

In [ ]:
if n_features == 1:
    fig1 = plt.figure(figsize=(15, 5))
    plt.ioff()
    x_plot = np.linspace(X.min() - 0.5, X.max() + 0.5, 1000)
    plt.plot(x_plot, [cost_median(theta) for theta in x_plot[:, None]])
    plt.plot(x_plot, [cost_huber(theta) for theta in x_plot[:, None]])
    plt.xlabel(r"$\theta$")
    plt.ylabel(r"$f(\theta)$")
    ymin = plt.ylim()[0]
    for x in X:
        plt.vlines(x, ymin=ymin, ymax=cost_median(x), linestyle='--', linewidth=1)
    #plt.scatter(X[:, 0], [cost_median(x, X) for x in X], marker='x')
    plt.show()

figname = "Smoothing4Median_visu"
my_saving_display(fig1, dirname, figname, imageformat, saving)

In [ ]:
theta = np.array([1.])
# return huber_prime(norm_X_m_theta).sum(axis=0)

In [ ]:
from scipy.optimize import minimize

# numerical minimization of the cost function, starting with the mean of the dataset:
opt_result = minimize(cost_median, x0=theta_init, method='BFGS')
true_median = opt_result["x"]
cost_min = cost_median(true_median)
print("scipy's solution: ", true_median)
print("scipy's objective: ", cost_min)


In [ ]:
L_const = 1. / thresh * n_samples
print("Rough Lipschitz constant: %.3f" % L_const)

list_gd_smooth, _ = gradient_descent(cost_huber, cost_huber_prime, steps=1. / L_const, theta_init=theta_init, n_iter=100)

In [ ]:
list_gd_smooth_obj=[cost_median(iterate) for iterate in list_gd_smooth]


fig1=plt.figure(figsize=(7,7))

plt.semilogy(list_gd_smooth_obj - cost_median(np.array([np.median(X)])),
             label=r"GD $\alpha={:.2f}$".format(thresh))
plt.xlabel("Number of epochs")
plt.ylabel(r"$f(\theta) -f(\theta^{\star})$")
plt.legend()

figname = "Smoothing4Median"
my_saving_display(fig1, dirname, figname, imageformat, saving)

In [ ]:
print("%.15f" % cost_min)

In [ ]:
print("%.15f" % cost_median(np.array([np.median(X)])))